In [ ]:
!pip install stumpy
!pip install finviz
!pip install nest-asyncio

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import stumpy
import pandas as pd
import finviz
import scipy
import pandas_datareader as pdr
import statsmodels
from sklearn import metrics
 
import timeit
import torch
import torch.nn as nn

from blitz.modules import BayesianLSTM
from blitz.modules import BayesianLinear
from blitz.utils import variational_estimator
 
from finviz.screener import Screener
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from statsmodels.api import OLS, add_constant
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import grangercausalitytests as gr_cause
 
def estimate_long_run_short_run_relationships(y, x):
    """Estimates long-run and short-run cointegration relationship for series y and x. 
    
    Uses a 2-step process to first estimate coefficients for the long-run relationship
        y_t = c + gamma * x_t + z_t
        
    and then the short-term relationship,
        y_t - y_(t-1) = alpha * z_(t-1) + epsilon_t,
        
    with z the found residuals of the first equation.
    
    Parameters
    ----------
    y : pd.Series
        The first time series of the pair to analyse.
    
    x : pd.Series
        The second time series of the pair to analyse.
    
    Returns
    -------
    c : float
        The constant term in the long-run relationship y_t = c + gamma * x_t + z_t. This 
        describes the static shift of y with respect to gamma * x.
        
    gamma : float
        The gamma term in the long-run relationship y_t = c + gamma * x_t + z_t. This
        describes the ratio between the const-shifted y and x.
        
    alpha : float
        The alpha term in the short-run relationship y_t - y_(t-1) = alpha * z_(t-1) + epsilon. This
        gives an indication of the strength of the error correction toward the long-run mean.
    
    z : pd.Series
        Series of residuals z_t from the long-run relationship y_t = c + gamma * x_t + z_t, representing
        the value of the error correction term. 
        
    """
    
    assert isinstance(y, pd.Series), 'Input series y should be of type pd.Series'
    assert isinstance(x, pd.Series), 'Input series x should be of type pd.Series'
    assert sum(y.isnull()) == 0, 'Input series y has nan-values. Unhandled case.'
    assert sum(x.isnull()) == 0, 'Input series x has nan-values. Unhandled case.'
    assert y.index.equals(x.index), 'The two input series y and x do not have the same index.'
    
    long_run_ols = OLS(y, add_constant(x), has_const=True)
    long_run_ols_fit = long_run_ols.fit()
    
    c, gamma = long_run_ols_fit.params
    z = long_run_ols_fit.resid
 
    short_run_ols = OLS(y.diff().iloc[1:], (z.shift().iloc[1:]))
    short_run_ols_fit = short_run_ols.fit()
    
    alpha = short_run_ols_fit.params[0]
        
    return c, gamma, alpha, z
 
 
def cointegrate(y, x,lag):
    """Applies the two-step Engle & Granger test for cointegration. 
    
    First fits the long-run relationship
        y_t = c + gamma * x_t + z_t
    
    and then tests, by Dickey-Fuller phi=1 vs phi < 1 in
        z_t = phi * z_(t-1) + eta_t
        
    If this implies phi < 1, the z series is stationary is concluded to be
    stationary, and thus the series y and x are concluded to be cointegrated.
    Parameters
    ----------
    y : pd.Series
        the first time series of the pair to analyse
    
    x : pd.Series
        the second time series of the pair to analyse
        
    Returns
    -------
    dfstat : float
        The Dickey Fuller test-statistic for phi = 1 vs phi < 1 in the second equation. A more 
        negative value implies the existence of stronger cointegration.
    
    pvalue : float
        The p-value corresponding to the Dickey Fuller test-statistic. A lower value implies
        stronger rejection of no-cointegration, thus stronger evidence of cointegration.
        
    """
    
    assert isinstance(y, pd.Series), 'Input series y should be of type pd.Series'
    assert isinstance(x, pd.Series), 'Input series x should be of type pd.Series'
    assert sum(y.isnull()) == 0, 'Input series y has nan-values. Unhandled case.'
    assert sum(x.isnull()) == 0, 'Input series x has nan-values. Unhandled case.'
    assert y.index.equals(x.index), 'The two input series y and x do not have the same index.'
    
    c, gamma, alpha, z = estimate_long_run_short_run_relationships(y, x)
    
    # NOTE: The p-value returned by the adfuller function assumes we do not estimate z first, but test 
    # stationarity of an unestimated series directly. This assumption should have limited effect for high N, 
    # so for the purposes of this course this p-value can be used for the EG-test. Critical values taking 
    # this into account more accurately are provided in e.g. McKinnon (1990) and Engle & Yoo (1987).
    
    adfstat, p_value, lags, nobs, crit_values = adfuller(z, maxlag=lag, autolag=None)
   
    return p_value, gamma,alpha,lags
 
 
def Coint_Mats(df):
  P_Mat=pd.DataFrame(index=df.columns,columns=df.columns)
  for stock1 in df:
    for stock2 in df:
      if stock1==stock2:
        P_Mat.loc[stock1,stock2]=0
      else:
        p_value,gamma,alpha,lags =cointegrate(df[stock1],df[stock2],lag=0)
        P_Mat.loc[stock1,stock2]=p_value
  return  P_Mat

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
def Naive_Subsequent(main_ticker):
  global close_df,vol_df,pair,period,query
  stock_info=finviz.get_stock(main_ticker)
  
  industry=''.join(e for e in stock_info['Industry'].lower() if e.isalnum())
  filters=['ind_'+industry]
  
  screener=Screener(filters=filters)
  
  ticker_list=[info['Ticker'] for info in screener]
  
  vol_df=pd.DataFrame()
  close_df=pd.DataFrame()
  for ticker in ticker_list:
    ticker=str(ticker) 
    close_df[ticker]=pdr.get_data_yahoo(ticker).Close.iloc[:-1]                       ######### modified
    vol_df[ticker]=(pdr.get_data_yahoo(ticker).Volume.iloc[:-1]+0.01).pct_change()    ########

  nulls=np.where(close_df.isnull().sum()>20)[0]
  close_df=close_df.drop(close_df.iloc[:,nulls],axis=1)
  pct_df=close_df.pct_change()
   
  coint_mat=Coint_Mats(close_df)
  
  idx,idy=np.where(coint_mat<=0.05)
  
  indices=np.column_stack([coint_mat.index[idx], coint_mat.columns[idy]])
  
  coint_pairs=[pair.tolist() for pair in indices if (coint_mat.loc[pair[0],pair[-1]] <=0.03) & (coint_mat.loc[pair[-1],pair[0]] <=0.03)]
  
  periods_list=[10,11,12,13,14,15,16,17,18,19,20] #1 and 2 weeks, 1 month 
  periods_df=pd.DataFrame()

  for period in periods_list:
    for pair in coint_pairs:
      try:
        query=pct_df[pair[0]].iloc[-period:]
        series=pct_df[pair[1]]
        dist_prof=stumpy.core.mass(query,series)

        short_prof=dist_prof

        top_n=6
        
        if len(short_prof)<=top_n:
          idxs=np.argpartition(short_prof,top_n)
        else:
          idxs = np.argpartition(short_prof,top_n)[:top_n]

        idxs = idxs[np.argsort(short_prof[idxs])]
    
        vol_error_list=[Naive_Volumes(pair,idx) for idx in idxs]
        
        #for idx in idxs:
          #vol_ratiosQ=vol_df[pair[0]].iloc[-period:]
          #vol_ratiosS=vol_df[pair[1]].iloc[idx:idx+len(query)] 
              
          #vol_error=metrics.mean_squared_error(vol_ratiosQ,vol_ratiosS)
          #vol_error_list.append(vol_error)


        lowest=np.argmin(vol_error_list)

        #print('{period} days: {stock1} prediction for next period from {stock2} is {prediction} with vol error {vol_error}'
        #.format(period=period,stock1=pair[0],stock2=pair[1],prediction=float(pct_df[pair[1]].iloc[idxs[lowest]+len(query)]),vol_error=vol_error_list[lowest]))
      
        if vol_error_list[lowest] <=1:
          prediction=float(pct_df[pair[1]].iloc[idxs[lowest]+len(query)+1])
          periods_df.loc[pair[0]+str(period),pair[1]]=prediction

      except (ValueError,IndexError):
        pass
    neg_scores=sum(n<0 for n in periods_df.values.flatten())
    pos_scores=sum(n>0 for n in periods_df.values.flatten())
    average_score=periods_df.mean().median()

    print(neg_scores,'neg scores',pos_scores,'pos scores',average_score,'is average')
    return periods_df

def Naive_Volumes(pair,idx):
  vol_ratiosQ=vol_df[pair[0]].iloc[-period:]
  vol_ratiosS=vol_df[pair[1]].iloc[idx:idx+len(query)] 
      
  vol_error=metrics.mean_squared_error(vol_ratiosQ,vol_ratiosS)
  #vol_error=abs(1-(metrics.r2_score(vol_ratiosQ,vol_ratiosS)))
  
  return vol_error

In [ ]:
def get_confidence_intervals(periods_df, ci_multiplier):
    
  #pred_mean=np.nanmean(periods_df.values)
  pred_mean=periods_df.mean().median()
  pred_std=np.nanstd(periods_df.values)
  
  upper_bound = pred_mean + (pred_std * ci_multiplier)
  lower_bound = pred_mean - (pred_std * ci_multiplier)
  #gather unscaled confidence intervals

  return pred_mean,upper_bound,lower_bound

In [ ]:
def confidence_acc(periods_df):
  pred_mean,upper_bound,lower_bound=get_confidence_intervals(periods_df,10)
  ic_acc=np.nanmean((lower_bound <=periods_df.values) * (upper_bound >=periods_df.values))
  under_ci_upper=np.nanmean(upper_bound >= periods_df.values).mean()
  over_ci_lower=np.nanmean(lower_bound <= periods_df.values)
  print("CI acc: {:.2f}, CI upper acc: {:.2f}, CI lower acc: {:.2f}".format(ic_acc, under_ci_upper, over_ci_lower))

  return ic_acc

In [ ]:
 def Position_Size(account_value,risk_percent,price,stop_loss): 
  account_risk=risk_percent*account_value
  trade_risk=price-stop_loss
  position_size=account_risk/trade_risk
  trade_risk_percent= trade_risk/price
 
  print('position_size:',position_size)
  
  return position_size,trade_risk,trade_risk_percent

In [ ]:
def DoEverything():
  string=input()
  ticker_list=string.split(',')

  for ticker in ticker_list:
    periods_df=Naive_Subsequent(ticker)

    ic_acc=confidence_acc(periods_df)

    print('ticker',':',ticker)

In [ ]:
DoEverything()

leu


100%|██████████| 1/1 [00:00<00:00,  8.63it/s]


15 neg scores 8 pos scores -0.011129530387807207 is average
CI acc: 0.72, CI upper acc: 0.72, CI lower acc: 0.72
ticker : leu
